In [1]:
import sys
import os 

WORK_AREA = "/Users/asaf/Workspace/biu/complex-utterance-to-code"
os.chdir(WORK_AREA)

paths = ['./src/', './src/api/v6', './notebooks/src',]
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [2]:
from typing import List, Union, Optional, TypeVar, Generic
import pandas as pd
import ast
import math
import glob
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory
import tokenize
from nltk.translate import bleu_score
from eval.utils import *

In [3]:
results_file_path = "~/Downloads/results/codet5-small-rep2rep-test-82-2023-05-17_133852.csv.gz"
results_df = pd.read_csv(results_file_path)
result = model_eval(
    results_df=results_df,
    output_column='output', 
    gold_column='code', 
    parse_to_code=True, 
    compute_humanval=True, 
    compute_bleu=False
)
result['humaneval']

{'score': 0.27261904761904765,
 'results':                            code_failure  execution_success   
 sample_id sample_minor_id                                    
 0         NaN                         0                  1  \
 1         a                           0                  1   
           b                           0                  1   
 2         NaN                         0                  0   
 3         a                           0                  1   
 ...                                 ...                ...   
 87        NaN                         0                  0   
 88        a                           0                  0   
           b                           0                  0   
 92        NaN                         0                  1   
 94        NaN                         0                  1   
 
                            execution_failuers  assertion_failure  correct   
 sample_id sample_minor_id                                 

In [4]:
result = model_eval(
    results_file_path="~/Downloads/results/codet5-small-rep2rep-test-82-2023-05-17_133852.csv.gz", 
    output_column='output', 
    gold_column='code', 
    parse_to_code=True, 
    compute_humanval=False, 
    compute_bleu=True
)
result['bleu']

{'score': 0.32169741117667017,
 'results':                            bleu_score
 sample_id sample_minor_id            
 0         NaN                0.000000
 1         a                  0.226174
           b                  0.226174
 2         NaN                0.186709
 3         a                  0.266707
 ...                               ...
 87        NaN                0.176574
 88        a                  0.480069
           b                  0.480069
 92        NaN                0.371473
 94        NaN                0.228220
 
 [82 rows x 1 columns]}

In [5]:
model_eval(
    results_file_path="~/Downloads/results/codet5-small-rep2rep-test-82-2023-05-17_133852.csv.gz", 
    output_column='output', 
    gold_column='code', 
    parse_to_code=True, 
    compute_humanval=True, 
    compute_bleu=False
)

{'humaneval': {'score': 0.27261904761904765,
  'results':                            code_failure  execution_failuers  correct   
  sample_id sample_minor_id                                              
  0         NaN                         0                   0        1  \
  1         a                           0                   0        0   
            b                           0                   0        0   
  2         NaN                         1                   0        0   
  3         a                           0                   0        2   
  ...                                 ...                 ...      ...   
  87        NaN                         1                   0        0   
  88        a                           1                   0        0   
            b                           1                   0        0   
  92        NaN                         0                   0        2   
  94        NaN                         0               

In [22]:
model_eval(
    results_file_path="~/Downloads/results/codet5-base-rep2rep-test-82-2023-05-18_161634.csv.gz",
    parse_to_code=True,
)

{'humaneval': {'score': 0.2374627976190476,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         1        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          2  0.000000
  3         a                           0        2          0  1.000000
  ...                                 ...      ...        ...       ...
  87        NaN                         0        0          2  0.000000
  88        a                           1        0          0  0.000000
            b                           1        0          0  0.000000
  92        NaN                         0        2          1  0.666667
  94        NaN                         0        0          2  0.000000
  
  [82 

In [23]:
model_eval(
    results_file_path="~/Downloads/results/codet5p-220m-rep2rep-test-82-2023-05-18_181859.csv.gz",
    parse_to_code=True,
)

{'humaneval': {'score': 0.28140219155844154,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         1        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          2  0.000000
  3         a                           0        1          1  0.500000
  ...                                 ...      ...        ...       ...
  87        NaN                         0        0          2  0.000000
  88        a                           0        3          2  0.600000
            b                           0        2          1  0.666667
  92        NaN                         0        2          1  0.666667
  94        NaN                         1        0          0  0.000000
  
  [82

In [24]:
model_eval(results_file_path = "~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz")

{'humaneval': {'score': 0.4106398809523809,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         1        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          2  0.000000
  3         a                           0        2          0  1.000000
  ...                                 ...      ...        ...       ...
  87        NaN                         0        2          1  0.666667
  88        a                           0        6          0  1.000000
            b                           0        0          2  0.000000
  92        NaN                         0        2          1  0.666667
  94        NaN                         0        0          2  0.000000
  
  [82 

In [25]:
model_eval(results_file_path = "~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz")

{'humaneval': {'score': 0.4106398809523809,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         1        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          2  0.000000
  3         a                           0        2          0  1.000000
  ...                                 ...      ...        ...       ...
  87        NaN                         0        2          1  0.666667
  88        a                           0        6          0  1.000000
            b                           0        0          2  0.000000
  92        NaN                         0        2          1  0.666667
  94        NaN                         0        0          2  0.000000
  
  [82 

In [6]:
model_eval(results_file_path = "~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz")

{'humaneval': {'score': 0.38590029761904765,
  'results':                            code_failure  execution_failuers  correct   
  sample_id sample_minor_id                                              
  0         NaN                         0                   0        3  \
  1         a                           0                   0        0   
            b                           0                   0        0   
  2         NaN                         0                   0        0   
  3         a                           0                   0        2   
  ...                                 ...                 ...      ...   
  87        NaN                         1                   0        0   
  88        a                           0                   0        6   
            b                           0                   0        0   
  92        NaN                         0                   0        2   
  94        NaN                         0               

In [26]:
df = pd.read_csv("~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz")
df.set_index('test_id', inplace=True)

In [27]:
test_id = '88_a'
print(df.loc[test_id]['text'])
print(df.loc[test_id]['output'])

Check if it's supposed to rain tonight and if it's not text Brian that I want to go out tonight
product_name = ProductName.resolve_from_text("rain")
product_attribute = ProductAttribute.resolve_from_text("tonight")
products = Shopping.find_products(product_name=product_name, product_attribute=product_attribute)
Responder.respond(response=products)

message_content_type = ContentType.resolve_from_text("text")
recipient = Contact.resolve_from_text("Brian")
content = Content.resolve_from_text("I want to go out tonight")
Message.send_message(recipient=recipient, content=content, message_content_type=message_content_type)


In [28]:
model_eval(
    results_file_path = "~/Downloads/results/codet5-small-text2rep-test-92-2023-05-21_012655.csv.gz",
    parse_to_code=True
)

{'humaneval': {'score': 0.22916666666666666,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         1        0          0  0.000000
  1         a                           1        0          0  0.000000
            b                           1        0          0  0.000000
  2         NaN                         1        0          0  0.000000
  3         a                           1        0          0  0.000000
  ...                                 ...      ...        ...       ...
  104       b                           1        0          0  0.000000
  105       NaN                         0        2          3  0.400000
  108       NaN                         0        0          2  0.000000
  109       a                           0        2          1  0.666667
            b                           0        1          0  1.000000
  
  [92

In [10]:
model_eval(
    results_file_path = "~/Downloads/results/codet5-small-rep2code-test-92-2023-05-21_012655.csv.gz",
    parse_to_code=True
)

{'humaneval': {'score': 0.16666666666666666,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         0        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          1  0.000000
  3         a                           0        0          1  0.000000
  ...                                 ...      ...        ...       ...
  104       b                           0        1          0  1.000000
  105       NaN                         0        0          1  0.000000
  108       NaN                         0        0          2  0.000000
  109       a                           0        2          1  0.666667
            b                           0        1          0  1.000000
  
  [92

In [18]:
model_eval(
    results_file_path = "~/Downloads/results/codet5-small-text2code-test-92-2023-05-21_012655.csv.gz",
    parse_to_code=False
)

{'humaneval': {'score': 0.2887152777777778,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         0        0          0  0.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         1        0          0  0.000000
  3         a                           0        2          0  1.000000
  ...                                 ...      ...        ...       ...
  104       b                           1        0          0  0.000000
  105       NaN                         1        0          0  0.000000
  108       NaN                         0        3          1  0.750000
  109       a                           0        1          2  0.333333
            b                           0        0          1  0.000000
  
  [92 

In [15]:
model_eval(
    results_file_path = "~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-20_233005.csv.gz",
    parse_to_code=False
)

{'humaneval': {'score': 0.25998883928571426,
  'results':                            code_failure  correct  incorrect  accuracy
  sample_id sample_minor_id                                            
  0         NaN                         0        2          0  1.000000
  1         a                           0        0          1  0.000000
            b                           0        0          1  0.000000
  2         NaN                         0        0          1  0.000000
  3         a                           0        2          0  1.000000
  ...                                 ...      ...        ...       ...
  87        NaN                         1        0          0  0.000000
  88        a                           0        3          1  0.750000
            b                           0        0          1  0.000000
  92        NaN                         1        0          0  0.000000
  94        NaN                         0        1          2  0.333333
  
  [82

In [14]:
results_df = pd.read_csv("~/Downloads/results/codet5-small-text_rep2rep-test-92-2023-05-21_012655.csv.gz")
model_eval(
    results_df = results_df,
    parse_to_code=True
)

{'humaneval': {'score': 0.1957561728395062,
  'results':                            code_failure  execution_failuers  correct   
  sample_id sample_minor_id                                              
  0         NaN                         1                   0        0  \
  1         a                           1                   0        0   
            b                           1                   0        0   
  2         NaN                         1                   0        0   
  3         a                           0                   0        2   
  ...                                 ...                 ...      ...   
  104       b                           1                   0        0   
  105       NaN                         1                   0        0   
  108       NaN                         1                   0        0   
  109       a                           0                   0        1   
            b                           0                

In [4]:
results_file_paths = glob.glob('dist/experiments_results/*')
results_file_paths = sorted(results_file_paths)
for results_file_path in results_file_paths:
    results_df = pd.read_csv(results_file_path)
    print('======================')
    print(results_file_path)
    results = model_eval(
        results_df=results_df,
        parse_to_code=('2rep' in results_file_path),
        compute_humanval=True, 
        compute_bleu=True
    )
    print(f"humaneval = {results['humaneval']['score']}")
    print(f"bleu = {results['bleu']['score']}")

dist/experiments_results/codet5-base-rep2rep-test-82-2023-05-18_161634.csv.gz
humaneval = 0.1015625
bleu = 0.32939804047533594
dist/experiments_results/codet5-small-rep2rep-test-82-2023-05-17_133852.csv.gz
humaneval = 0.08984375
bleu = 0.32208960846327794
dist/experiments_results/codet5-small-rep2rep-test-92-refit_complex_codet5-small-rep2rep-2023-05-23_031926.csv.gz
humaneval = 0.08159722222222222
bleu = 0.34448553541799093
dist/experiments_results/codet5p-220m-rep2code-test-82-2023-05-19_195010.csv.gz
humaneval = 0.1171875
bleu = 0.39721567752757525
dist/experiments_results/codet5p-220m-rep2rep-test-82-2023-05-18_181859.csv.gz
humaneval = 0.10546875
bleu = 0.3184125324573982
dist/experiments_results/codet5p-220m-rep2rep-test-92-2023-05-19_013955.csv.gz
humaneval = 0.10069444444444445
bleu = 0.3165757517735639
dist/experiments_results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz
humaneval = 0.12109375
bleu = 0.4219072151962306
dist/experiments_results/codet5p-220m-text2rep-

In [5]:
results_file_paths = glob.glob('/Users/asaf/Downloads/results3/*')
results_file_paths = sorted(results_file_paths)
for results_file_path in results_file_paths:
    results_df = pd.read_csv(results_file_path)
    print('======================')
    print(results_file_path)
    results = model_eval(
        results_df=results_df,
        parse_to_code=('2rep' in results_file_path),
        compute_humanval=True, 
        compute_bleu=False
    )
    print(f"humaneval = {results['humaneval']['score']}")
#     print(f"bleu = {results['bleu']['score']}")

/Users/asaf/Downloads/results3/test-152-codet5-base-rep2code-2023-05-25_131404.csv.gz
humaneval = 0.08367486338797815
/Users/asaf/Downloads/results3/test-152-codet5-base-rep2rep-2023-05-24_122620.csv.gz
humaneval = 0.0969945355191257
/Users/asaf/Downloads/results3/test-152-codet5-base-text2code-2023-05-25_125337.csv.gz
humaneval = 0.054098360655737705
/Users/asaf/Downloads/results3/test-152-codet5-base-text2rep-2023-05-24_143609.csv.gz
humaneval = 0.0633879781420765
/Users/asaf/Downloads/results3/test-152-codet5-base-text_rep2rep-2023-05-26_072118.csv.gz
humaneval = 0.06864754098360656
/Users/asaf/Downloads/results3/test-152-codet5-small-rep2code-2023-05-24_122619.csv.gz
humaneval = 0.06420765027322405
/Users/asaf/Downloads/results3/test-152-codet5-small-rep2rep-2023-05-23_140209.csv.gz
humaneval = 0.10314207650273223
/Users/asaf/Downloads/results3/test-152-codet5-small-text2code-2023-05-24_164951.csv.gz
humaneval = 0.06967213114754098
/Users/asaf/Downloads/results3/test-152-codet5-sma

In [8]:
results_file_paths = glob.glob('/Users/asaf/Downloads/results3/*')
results_file_paths = sorted(results_file_paths)
for results_file_path in results_file_paths:
    results_df = pd.read_csv(results_file_path)
    print('======================')
    print(results_file_path)
    results = model_eval(
        results_df=results_df,
        parse_to_code=('2rep' in results_file_path),
        compute_humanval=True, 
        compute_bleu=True
    )
#     print(f"humaneval = {results['humaneval']['results']}")
    stats = results['humaneval']['results'][['code_failure', 'execution_success', 'execution_failure', 'assertion_failure', 'correct', 'incorrect']].sum()
    x = stats['execution_success'] / (stats['execution_success'] + stats['execution_failure'])
    print(x)
    print(stats)
#     print(f"bleu = {results['bleu']['score']}")

/Users/asaf/Downloads/results3/test-152-codet5-base-rep2code-2023-05-25_131404.csv.gz
0.45394736842105265
code_failure           0
execution_success     69
execution_failure     83
assertion_failure      0
correct              124
incorrect             80
dtype: int64
/Users/asaf/Downloads/results3/test-152-codet5-base-rep2rep-2023-05-24_122620.csv.gz
0.5328947368421053
code_failure           0
execution_success     81
execution_failure     71
assertion_failure      0
correct              125
incorrect             90
dtype: int64
/Users/asaf/Downloads/results3/test-152-codet5-base-text2code-2023-05-25_125337.csv.gz
0.40131578947368424
code_failure           0
execution_success     61
execution_failure     91
assertion_failure      0
correct              103
incorrect             76
dtype: int64
/Users/asaf/Downloads/results3/test-152-codet5-base-text2rep-2023-05-24_143609.csv.gz
0.4605263157894737
code_failure           0
execution_success     70
execution_failure     82
assertion_fail

In [14]:
results['humaneval']['results'][['code_failure', 'execution_failuers', 'correct', 'incorrect']].sum()

code_failure          152
execution_failuers      0
correct                 0
incorrect               0
dtype: int64